In [4]:
import numpy as np
from my_ueg import my_ueg, my_ueg_faster
from pyscf import gto, scf, cc
import time

In [6]:
nocc = 7 # 7, 19, 27, 33, 57, 81, 93
t0 = time.time()
my_sys = my_ueg(rs=1.0, nelec=(nocc, nocc), gamma=3) #np.sqrt(2))
print('num kpts: ',my_sys.nkpts)
h0 = my_sys.madelung() / 2
h1 = my_sys.get_h1_real()
cderi = my_sys.get_cderi_real()
print('cderi shape: ', cderi.shape)
t1 = time.time()
print('time build cderi: ', t1-t0)

mol = gto.M()
mol.nelectron = my_sys.nparticle
mol.nao = my_sys.nkpts
mol.incore_anyway = True
mol.max_memory = 20000
mol.verbose = 4

dm = np.zeros((my_sys.nkpts,my_sys.nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol).density_fit()
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(my_sys.nkpts)
mf._cderi = cderi
mf.init_guess = "1e"
mf.max_cycle = -1
escf = mf.kernel(dm0=dm)

mf.mo_coeff = np.eye(my_sys.nkpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/my_sys.nparticle}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
mycc.conv_tol = 1e-7
mycc.kernel()

et = mycc.ccsd_t()
print("CCSD energy", mycc.e_corr)
print("CCSD energy per electron", mycc.e_corr / my_sys.nparticle)
print("CCSD(T) energy", et)
print("CCSD(T) energy per electron", et / my_sys.nparticle)

num kpts:  179
cderi shape:  (1172, 16110)
time build cderi:  72.34545016288757


******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmpc4kj7l0n
max_memory 20000 MB (current use 443 MB)
Set gradient conv threshold to 3.16228e-05
init E= 8.49148146743777
  HOMO = 1.04145818327265  LUMO = 2.32324526529524
SCF not converged.
SCF energy = 8.49148146743777
escf: 8.491481467437769
escf per electron: 0.6065343905312692

******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 7, nmo = 179
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 459 MB)


Overwritten attributes  energy_nuc get_ovlp get_hcore  of <class 'pyscf.df.df_jk.DFRHF'>


Init t2, MP2 energy = 7.82577463544202  E_corr(MP2) -0.665706831995745
Init E_corr(RCCSD) = -0.665706831995721
cycle = 1  E_corr(RCCSD) = -0.425628685913301  dE = 0.240078146  norm(t1,t2) = 0.116877
cycle = 2  E_corr(RCCSD) = -0.539422426171114  dE = -0.11379374  norm(t1,t2) = 0.0586917
cycle = 3  E_corr(RCCSD) = -0.502466817664043  dE = 0.0369556085  norm(t1,t2) = 0.0289294
cycle = 4  E_corr(RCCSD) = -0.502490529828804  dE = -2.37121648e-05  norm(t1,t2) = 0.000163516
cycle = 5  E_corr(RCCSD) = -0.502516763021458  dE = -2.62331927e-05  norm(t1,t2) = 2.5161e-05
cycle = 6  E_corr(RCCSD) = -0.502519527962316  dE = -2.76494086e-06  norm(t1,t2) = 2.75462e-06
cycle = 7  E_corr(RCCSD) = -0.502519704717114  dE = -1.76754798e-07  norm(t1,t2) = 1.12831e-07
cycle = 8  E_corr(RCCSD) = -0.502519671788051  dE = 3.2929063e-08  norm(t1,t2) = 2.71476e-08
RCCSD converged
E(RCCSD) = 7.988961795649718  E_corr = -0.5025196717880507
RCCSD(T) correction = -0.0199468653388013
CCSD energy -0.5025196717880507
C

In [7]:
nocc = 7 # 7, 19, 27, 33, 57, 81, 93
t0 = time.time()
my_sys = my_ueg_faster(rs=1.0, nelec=(nocc, nocc), gamma=3) #np.sqrt(2))
print('num kpts: ',my_sys.nkpts)
h0 = my_sys.madelung() / 2
h1 = my_sys.get_h1_real()
cderi = my_sys.get_cderi_real()
print('cderi shape: ', cderi.shape)
t1 = time.time()
print('time build cderi: ', t1-t0)

mol = gto.M()
mol.nelectron = my_sys.nparticle
mol.nao = my_sys.nkpts
mol.incore_anyway = True
mol.max_memory = 20000
mol.verbose = 4

dm = np.zeros((my_sys.nkpts,my_sys.nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol).density_fit()
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(my_sys.nkpts)
mf._cderi = cderi
mf.init_guess = "1e"
mf.max_cycle = -1
escf = mf.kernel(dm0=dm)

mf.mo_coeff = np.eye(my_sys.nkpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/my_sys.nparticle}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
mycc.conv_tol = 1e-7
mycc.kernel()

et = mycc.ccsd_t()
print("CCSD energy", mycc.e_corr)
print("CCSD energy per electron", mycc.e_corr / my_sys.nparticle)
print("CCSD(T) energy", et)
print("CCSD(T) energy per electron", et / my_sys.nparticle)

num kpts:  179
cderi shape:  (1172, 16110)
time build cderi:  2.134516477584839


******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmp2mz_h7mt
max_memory 20000 MB (current use 911 MB)
Set gradient conv threshold to 3.16228e-05
init E= 8.49148146743777
  HOMO = 1.04145818327265  LUMO = 2.32324526529524
SCF not converged.
SCF energy = 8.49148146743777
escf: 8.491481467437769
escf per electron: 0.6065343905312692

******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 7, nmo = 179
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 601 MB)
Init t2, MP2 energy = 7.82577463544202  E_co

In [5]:
print(my_sys.kpts)

[[0. 0. 0.]
 [0. 0. 0.]]


In [4]:
my_sys.prep_afqmc(mycc)

# Size of the correlation space:
# Number of electrons: (7, 7)
# Number of basis (k-points): 19
# Number of CholVecs (q-points): 92
